## Feature Engineering in RAM-Limited Data, Part 5

#### CV for mean encoding of `target_combine`

#### CV@2015-12-28:
- benchmark: val = 1.62857
- with only `ind_actividad_client_combine`, `tiprel_1mes_combine`, `target_combine`, mlogloss=1.57141
- with `ind_actividad_client_combine`, `tiprel_1mes_combine`, `target_combine`, `n_products` and patterns: val = 1.31122
    - Private score: 0.0302475, public score: 0.0299266

- with all above and mean encoding of target indicator and target #products: mlogloss=1.30756
    - Private score: 0.0302597, public score: 0.0299519

In [1]:
from santander_helper import *
%matplotlib inline

Generate data

In [2]:
x_train, y_train = create_train_test('2015-06-28', target_flag=True, pattern_flag=True)

Loading 2015-05-28 data
Loading 2015-06-28 data
Products in 2015-06-28
Products in 2015-05-28
New products added in 2015-06-28
Join customer features and previous month products for 2015-06-28
Concatenate this and previous months ind_activadad_cliente
Concatenate this and previous months tiprel_1mes
Combine all products for each customer

Start counting patterns:
Encoding pattern...


100%|████████████████████████████████████████████████████████████████████████| 640825/640825 [01:42<00:00, 6249.66it/s]


Finding pattern...
Counting pattern...


100%|████████████████████████████████████████████████████████████████████████| 634760/634760 [03:13<00:00, 3286.08it/s]


Prepare target
Time used: 6.167 min


In [3]:
x_val, y_val = create_train_test('2015-12-28', max_lag=5, target_flag=True, pattern_flag=True)

Loading 2015-11-28 data
Loading 2015-12-28 data
Products in 2015-12-28
Products in 2015-11-28
New products added in 2015-12-28
Join customer features and previous month products for 2015-12-28
Concatenate this and previous months ind_activadad_cliente
Concatenate this and previous months tiprel_1mes
Combine all products for each customer

Start counting patterns:
Encoding pattern...


100%|████████████████████████████████████████████████████████████████████████| 912728/912728 [02:22<00:00, 6398.12it/s]


Finding pattern...
Counting pattern...


100%|████████████████████████████████████████████████████████████████████████| 893990/893990 [04:29<00:00, 3318.78it/s]


Prepare target
Time used: 8.612 min


In [4]:
x_test = create_train_test('2016-06-28', max_lag=5, target_flag=False, pattern_flag=True)

Loading 2016-05-28 data
Loading 2016-06-28 data
Products in 2016-06-28
Products in 2016-05-28
New products added in 2016-06-28
Join customer features and previous month products for 2016-06-28
Concatenate this and previous months ind_activadad_cliente
Concatenate this and previous months tiprel_1mes
Combine all products for each customer

Start counting patterns:
Encoding pattern...


100%|████████████████████████████████████████████████████████████████████████| 938423/938423 [02:28<00:00, 6321.45it/s]


Finding pattern...
Counting pattern...


100%|████████████████████████████████████████████████████████████████████████| 930436/930436 [04:41<00:00, 3300.62it/s]


Time used: 7.568 min


## Train model

- benchmark: val = 1.62857
- with only `ind_actividad_client_combine`, `tiprel_1mes_combine`, `target_combine`, mlogloss=1.57141
- with `ind_actividad_client_combine`, `tiprel_1mes_combine`, `target_combine`, `n_products` and patterns: val = 1.31122
- with all above and mean encoding of target indicator and target #products: mlogloss=1.30756
- with all above and mean encoding of each product: mlogloss=1.29115

In [5]:
param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}
num_rounds = 50

dtrain = xgb.DMatrix(x_train.values, y_train.values)
dval = xgb.DMatrix(x_val.values, y_val.values)
model = xgb.train(param, dtrain, num_rounds, evals=[(dtrain, 'train'), (dval, 'val')], verbose_eval=True)

[0]	train-mlogloss:2.70789	val-mlogloss:2.73135
[1]	train-mlogloss:2.536	val-mlogloss:2.57759
[2]	train-mlogloss:2.39973	val-mlogloss:2.45778
[3]	train-mlogloss:2.2849	val-mlogloss:2.35812
[4]	train-mlogloss:2.18635	val-mlogloss:2.27274
[5]	train-mlogloss:2.0994	val-mlogloss:2.19884
[6]	train-mlogloss:2.02227	val-mlogloss:2.13235
[7]	train-mlogloss:1.95316	val-mlogloss:2.07493
[8]	train-mlogloss:1.89008	val-mlogloss:2.02052
[9]	train-mlogloss:1.83238	val-mlogloss:1.97003
[10]	train-mlogloss:1.77957	val-mlogloss:1.9238
[11]	train-mlogloss:1.7306	val-mlogloss:1.88683
[12]	train-mlogloss:1.68535	val-mlogloss:1.84988
[13]	train-mlogloss:1.6435	val-mlogloss:1.81391
[14]	train-mlogloss:1.60427	val-mlogloss:1.78027
[15]	train-mlogloss:1.56761	val-mlogloss:1.75104
[16]	train-mlogloss:1.53337	val-mlogloss:1.72214
[17]	train-mlogloss:1.50111	val-mlogloss:1.69432
[18]	train-mlogloss:1.47095	val-mlogloss:1.66778
[19]	train-mlogloss:1.4421	val-mlogloss:1.64345
[20]	train-mlogloss:1.41485	val-mloglo

Prediction from my model

In [6]:
preds = model.predict(xgb.DMatrix(x_test.values))

df_preds = pd.DataFrame(preds, index=x_test.index, columns=target_cols)
# Remove already bought products 
df_preds[x_test[target_cols]==1] = 0 
preds = df_preds.values
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:, :7]

Write out prediction results from my model

In [7]:
test_id = x_test.loc[:, 'ncodpers'].values
final_preds = [' '.join([target_cols[k] for k in pred]) for pred in preds]

out_df = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds})
out_df.to_csv('eda_4_19.csv.gz', compression='gzip', index=False)

In [8]:
#pd.Series(x_train.columns).to_csv('x_train_cols.csv')